In [ ]:
%reload_ext dotenv
%dotenv
%reload_ext autoreload
%autoreload 2

In [ ]:
import replicate
import requests
from PIL import Image
from tqdm.notebook import tqdm

In [ ]:
humans = {
    "model": {"name": "model", "image_path": "../images/humans/model.jpg"},
    "nimo": {"name": "nimo", "image_path": "../images/humans/nimo_underwear.jpg"},
}
garments = {
    "tshirt": {
        "name": "tshirt",
        "description": "purple t-shirt",
        "category": "upper_body",
        "image_path": "../images/garments/tops/tshirt.webp",
    },
    "sweater": {
        "name": "sweater",
        "description": "oversized pink sweater",
        "category": "upper_body",
        "image_path": "../images/garments/tops/sweater.jpg",
    },
    "striped_sweater": {
        "name": "striped_sweater",
        "description": "black and white striped sweater",
        "category": "upper_body",
        "image_path": "../images/garments/tops/striped_sweater.webp",
    },
    "winter_coat": {
        "name": "winter_coat",
        "description": "winter coat with fur lined hood",
        "category": "upper_body",
        "image_path": "../images/garments/tops/winter_coat.webp",
    },
    "raincoat": {
        "name": "raincoat",
        "description": "light blue hip-length raincoat",
        "category": "upper_body",
        "image_path": "../images/garments/tops/raincoat.webp",
    },
    "jeans": {
        "name": "jeans",
        "description": "slim fit washed jeans",
        "category": "lower_body",
        "image_path": "../images/garments/bottoms/jeans.webp",
    },
    "joggers": {
        "name": "joggers",
        "description": "pink joggers",
        "category": "lower_body",
        "image_path": "../images/garments/bottoms/joggers.jpg",
    },
    "gym_shorts": {
        "name": "gym_shorts",
        "description": "short white gym shorts",
        "category": "lower_body",
        "image_path": "../images/garments/bottoms/gym_shorts.webp",
    }
}

In [ ]:
from sqlmodel import SQLModel, Session

from wardrobe.db import engine
from wardrobe.models import Wearable, WearableImage


with Session(engine) as session:
    for garment in garments.values():
        with open(garment["image_path"], "rb") as image_file:
            wearable_image = WearableImage(image_data=image_file.read())
            session.add(wearable_image)

        wearable = Wearable(
            category=garment["category"],
            description=garment["description"],
            wearable_image_id=wearable_image.id,
        )
        session.add(wearable)
    session.commit()

In [ ]:
from sqlmodel import Session, select
from wardrobe.db import engine
from wardrobe.models import WearableImage

with Session(engine) as session:
    session.add(WearableImage(image_data=b"foo"))
    u = session.exec(select(WearableImage).where(WearableImage.image_data == b"foo")).one()
    print("Found:", u)

### Figure out where to draw the garment (pre-masking)

Only need to do this once per category/human combination

In [ ]:
def generate_mask(category, human):
    input = {
        # Garment doesn't matter if mask_only is True
        "garm_img": open(next(iter(garments.values()))["image_path"], "rb"),
        "human_img": open(human["image_path"], "rb"),
        "category": category,
        "mask_only": True,
    }

    result_url = replicate.run(
        "cuuupid/idm-vton:c871bb9b046607b680449ecbae55fd8c6d945e0a1948644bf2361b3d021d3ff4",
        input=input,
    )
    
    return Image.open(requests.get(result_url, stream=True).raw)

In [ ]:
im = generate_mask(category="lower_body", human=humans["nimo"])
im.save(f"../images/masks/{humans["nimo"]["name"]}/pre/bottom.jpg")
im

### Put single garment on human

Needs a pre-mask for the given category/human combination

In [ ]:
def put_garment(garment, human):
    input = {
        "garm_img": open(garment["image_path"], "rb"),
        "human_img": open(human["image_path"], "rb"),
        "mask_img": open(
            f"../images/masks/{human["name"]}/pre/top.jpg"
            if garment["category"] == "upper_body"
            else f"../images/masks/{human["name"]}/pre/bottom.jpg",
            "rb",
        ),
        "garment_des": garment["description"],
        "category": garment["category"],
    }

    result_url = replicate.run(
        "cuuupid/idm-vton:c871bb9b046607b680449ecbae55fd8c6d945e0a1948644bf2361b3d021d3ff4",
        input=input,
    )

    return Image.open(requests.get(result_url, stream=True).raw)

In [ ]:
garment = garments["jeans"]
human = humans["nimo"]

im = put_garment(garment, human)
im.save(f"../images/results/{human["name"]}/single/{garment["name"]}.jpg")
im

In [ ]:
human = humans["nimo"]
for garment in tqdm(garments.values()):
    im = put_garment(garment, human)
    im.save(f"../images/results/{human["name"]}/single/{garment["name"]}.jpg")

### Find out where the top garment was drawn (post-masking)

Needs to be done for every top garment/human combination

In [ ]:
def mask_garment(garment, human):
    prompt = garment["description"]
    print(f"prompt: {repr(prompt)}")

    negative_prompt = "underwear, shoes"
    if garment["category"] == "upper_body":
        negative_prompt = "pants, " + negative_prompt
    else:
        negative_prompt = "top, " + negative_prompt
    print(f"negative_prompt: {repr(negative_prompt)}")


    input = {
        "image": open(f"../images/results/{human["name"]}/single/{garment["name"]}.jpg", "rb"),
        "mask_prompt": prompt,
        "negative_mask_prompt": negative_prompt,
        "adjustment_factor": 0
    }

    results = replicate.run(
        "schananas/grounded_sam:ee871c19efb1941f55f66a3d7d960428c8a5afcb77449547fe8e5a3ab9ebc21c",
        input=input
    )
    for result in results:
        print(f"result: {repr(result)}")
        # Results contains some other stuff, we only want the regular mask
        if result.endswith("/mask.jpg"):
            return Image.open(requests.get(result, stream=True).raw)
    raise ValueError("Could not get mask URL")

In [ ]:
garment = garments["striped_sweater"]
human = humans["nimo"]

im = mask_garment(garment, human)
im.save(f"../images/masks/{human["name"]}/post/{garment["name"]}.jpg")
im

In [ ]:
human = humans["nimo"]
for garment in tqdm(garments.values()):
    im = mask_garment(garment, human)
    im.save(f"../images/masks/{human["name"]}/post/{garment["name"]}.jpg")

### Combine garments

In [ ]:
from combining import combine_garments

human = humans["nimo"]
garment_top = garments["striped_sweater"]
garment_bottom = garments["jeans"]

human_im = Image.open(human["image_path"])
print(human_im.format, human_im.size, human_im.mode)

result_top_im = Image.open(
    f'../images/results/{human["name"]}/single/{garment_top["name"]}.jpg'
)
print(result_top_im.format, result_top_im.size, result_top_im.mode)

result_bottom_im = Image.open(
    f'../images/results/{human["name"]}/single/{garment_bottom["name"]}.jpg'
)
print(result_bottom_im.format, result_bottom_im.size, result_bottom_im.mode)

mask_top_im = Image.open(
    f'../images/masks/{human["name"]}/post/{garment_top["name"]}.jpg'
).convert("L")
print(mask_top_im.format, mask_top_im.size, mask_top_im.mode)

im = combine_garments(human_im, result_top_im, result_bottom_im, mask_top_im)
im.save(f"../images/results/{human["name"]}/multi/{garment_top["name"]}_{garment_bottom["name"]}.jpg")
im